In [ ]:
from matplotlib.cbook import contiguous_regions
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from pandas.core.sorting import get_compressed_ids
from pandas_ods_reader import read_ods
from scipy.optimize import curve_fit
from lab_lib import *

In [ ]:
cal_file = read_ods("data/calibration.ods").to_numpy()

lambd = [i[0] for i in cal_file]
theta = [i[1] for i in cal_file]

def disp_func(x, a, b, c):
    return a + b / (x - c)

cal_func, cal_coeffs = make_curve_approx("img/calibration.png", r"Калибровочный график $\lambda = f(\theta)$", r"$\theta$", r"$\lambda, $\AA", theta, lambd, [2]*len(lambd), [0]*len(lambd), disp_func, p0=(4000, -700*10**4, 4000))

for coeff in cal_coeffs:
    print(texAns(coeff))

cal_table = [[r"$\theta$", r"$\lambda, $\AA"]]
for i in range(len(lambd)):
    cal_table.append([texAns((theta[i], 2)), str(int(lambd[i]))])

texTable(cal_table)

In [ ]:
file = read_ods(f"data/H.ods").to_numpy()

names = [r"$H_{\alpha}$", r"$H_{\beta}$", r"$H_{\gamma}$", r"$H_{\delta}$"]
ethalon = [6563, 4861, 4341, 4102]
ms = [3, 4, 5, 6]
thetas = [(i, 2) for i in file[0]]

def proc_H(theta, m, n=2):
    lambd = cal_func(theta)
    #lambd = (lambd[0], np.sqrt(lambd[1]**2 + (cal_coeffs[1][1] / (theta[0] - cal_coeffs[2][0]))**2))

    R = (1/lambd[0] * m**2 * n**2 / (m**2 - n**2), lambd[1]/lambd[0]**2)

    return lambd, R

lambdas = []
Rs = []
for theta, m in zip(thetas, ms):
    lambd, R = proc_H(theta, m)
    lambdas.append(lambd)
    Rs.append(R)

H_table = [[""] + names,
           [r"$\theta$"] + [texAns(i) for i in thetas],
           [r"$\lambda, $\AA"] + [texAns(i) for i in lambdas],
           [r"$\lambda_0, $\AA"] + [texAns((i,)) for i in ethalon],
           [r"$R, см^{-1}$"] + [texAns(tuple(j*10**8 for j in i)) for i in Rs]]

texTable(H_table)

R_mean = meanWithInacc(Rs)

print(r"$<R> = $", texAns(tuple(i*10**8 for i in R_mean)), r"$см^{-1}$")

In [ ]:
file = read_ods(f"data/I.ods").to_numpy()

thetas = [(i[1], 2) for i in file]
theta_max = (file[0][2], 2)

lambdas = [cal_func(theta) for theta in thetas]
lambd_max = cal_func(theta_max)

print(texAns(lambdas[0]))
print(texAns(lambdas[-1]))
print(texAns(lambd_max))

h = 6.6261 * 10**-27
c = 2.9979 * 10**10
def get_E(lambd):
    return (h * c / lambd[0] * 10**8 * 6.242*10**11, h * c * lambd[1] / lambd[0]**2 * 10**8 * 6.242*10**11)

Es = [get_E(lambd) for lambd in lambdas]

E_2 = ((Es[-1][0] - Es[0][0]) / (len(Es) - 1), np.sqrt(Es[-1][1]**2 + Es[0][1]**2) / (len(Es) - 1))

print(r"$h\nu_2 = $", texAns(E_2), r"$эВ$")

E_max = get_E(lambd_max)

print(r"$h\nu_{гр} = $", texAns(E_max), r"$эВ$")

E_1 = 0.027
E_a = 0.94

E_el = (Es[0][0] - 1/2*E_2[0] + 3/2*E_1, np.sqrt(Es[0][1]**2 + 1/4*E_2[1]**2))

print(r"$h\nu_{эл} = $", texAns(E_el), r"$эВ$")

D_1 = (E_max[0] - E_a, E_max[1])

print(r"$D_1 = $", texAns(D_1), r"$эВ")

D_2 = (E_max[0] - E_el[0], np.sqrt(E_max[1]**2 + E_el[1]**2))

print(r"$D_2 = $", texAns(D_2), r"$эВ")